In [7]:
from utils.SQLutils import connectMysql, readMysqlTable2DF, MysqlEngine
import featureUtils
import pandas as pd
import numpy as np

# 连接Mysql的引擎
sql_engine = MysqlEngine(host='localhost',port='3306',user='wieneralan',password='0922',database='stock_data')
## 从Mysql的中读数据
symbols = readMysqlTable2DF(sql_engine,'hs300name_wiki', sel_column='code')
df = readMysqlTable2DF(sql_engine,'hs300_daily_price', sel_column='*')
print('hs300 shape ',df.shape)
# 定义特征数据列表
df_feat = pd.DataFrame()
                        
# 循环提取特征
num = 0



hs300 shape  (1078154, 15)


In [8]:
for index, symbol in symbols['code'].items():
    num+=1
    if(num>2):
        break
    # 依次取出每个股票的数据
    iter_feat = pd.DataFrame() # 初始化每次的数据列表
    df_symbol=df.query('symbol == @symbol')
    # 增加股票id和日期列
    iter_feat['symbol'] = df_symbol['symbol'] # 存股票id
    iter_feat['price_date'] = df_symbol['price_date'] # 存股票日期
    iter_feat['close_price'] = df_symbol['close_price'] # 存股票日期

    # 计算特征
    
    iter_feat['open_price_daily_return'] = featureUtils.create_open_price_return(df_symbol['open_price']) # 开盘价涨幅比例
    iter_feat['high_price_daily_return'] = featureUtils.create_history_return(df_symbol['high_price']) # 昨天最高价涨幅
    iter_feat['low_price_daily_return'] = featureUtils.create_history_return(df_symbol['low_price']) # 昨天最低价涨幅
    iter_feat['amount_daily_return'] = featureUtils.create_history_return(df_symbol['amount']) # 昨天成交量涨幅
    iter_feat['open_price_return_n'] = featureUtils.create_open_price_return_n(
                                        df_symbol['open_price'],df_symbol['high_price'],df_symbol['low_price'],5) # 今日相比昨日开盘的涨幅/过去n天最高价与最低价的差值
    iter_feat['EMV'] = featureUtils.create_EMV(df_symbol['high_price'], df_symbol['low_price'], df_symbol['amount'],14)

    iter_feat['SMA'] = featureUtils.create_SMA(df_symbol['close_price'], 4)
    # iter_feat['TEMA'] = featureUtils.create_TEMA(df_symbol['close_price'], 30).shift(1)
    # iter_feat['TRIMA'] = featureUtils.create_TRIMA(df_symbol['close_price'], 30).shift(1)
    # iter_feat['WMA'] = featureUtils.create_WMA(df_symbol['close_price'], 30).shift(1)
    # iter_feat['ROC'] = featureUtils.create_ROC(df_symbol['close_price'], 10).shift(1)
    # iter_feat['ROCP'] = featureUtils.create_ROCP(df_symbol['close_price'], 10).shift(1)
    # iter_feat['CCI'] = featureUtils.create_CCI(df_symbol['high_price'],df_symbol['low_price'],df_symbol['close_price'],20).shift(1)
    # iter_feat['Force_Index'] = featureUtils.create_Force_Index(df_symbol['close_price'], df_symbol['amount'], 1).shift(1)
    # iter_feat['BBupper'],iter_feat['middle2'],iter_feat['lower2'] = featureUtils.create_BBANDS(df_symbol['close_price'])
    
    # 存入DataFrame中
    df_feat = pd.concat([df_feat, iter_feat])
    print(num)

1
2


In [3]:
print(df_feat)

       symbol price_date  close_price  open_price_daily_return  \
0      000001 1991-04-03        49.00                      NaN   
1      000001 1991-04-04        48.76                -0.004898   
2      000001 1991-04-05        48.52                -0.004922   
3      000001 1991-04-06        48.28                -0.004946   
4      000001 1991-04-08        48.04                -0.004971   
...       ...        ...          ...                      ...   
15610  000002 2023-11-21        11.92                 0.006832   
15611  000002 2023-11-22        11.84                 0.005089   
15612  000002 2023-11-23        12.17                 0.000844   
15613  000002 2023-11-24        12.12                 0.027825   
15614  000002 2023-11-27        11.80                -0.010664   

       high_price_daily_return  low_price_daily_return  amount_daily_return  \
0                          NaN                     NaN                  NaN   
1                          NaN                   

In [4]:
print(df_feat[close_price],df_feat[SMA])

NameError: name 'close_price' is not defined

In [10]:
print(df_feat["close_price"][:15],df_feat["SMA"][:15])

0     49.00
1     48.76
2     48.52
3     48.28
4     48.04
5     47.80
6     47.56
7     47.08
8     46.84
9     46.38
10    46.15
11    45.92
12    45.69
13    45.46
14    45.00
Name: close_price, dtype: float64 0         NaN
1         NaN
2         NaN
3     48.6400
4     48.4000
5     48.1600
6     47.9200
7     47.6200
8     47.3200
9     46.9650
10    46.6125
11    46.3225
12    46.0350
13    45.8050
14    45.5175
Name: SMA, dtype: float64
